In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Standard imports
import numpy as np
import xarray as xr
import tqdm as tqdm

In [3]:
# Import OpenSense modules as submodules
import sys
import os

sys.path.append(os.path.abspath("./pycomlink/"))
sys.path.append(os.path.abspath("./poligrain/src/"))
sys.path.append(os.path.abspath("./mergeplg/src/"))

import pycomlink as pycml 
import poligrain as plg
import mergeplg 

In [4]:
os.makedirs('data/adjusted_fields', exist_ok=True)

# OpenMRG adjustment

In [5]:
# OpenMRG
ds_rad = xr.open_dataset("data/andersson_2022_OpenMRG/radar/openmrg_rad.nc")                    
ds_cmls = xr.open_dataset("data/processed_cml_OpenMRG.nc").R_acc       
months = ['2015-06', '2015-07', '2015-08']                           

In [6]:
# methods parameter : default version
nnear = 12 
variogram_parameters = {"sill": 1.0, "range": 30000, "nugget": 0.3}
diff_check_sel = 10
ratio_check_sel = (0.1,15)

In [7]:
# additive IDW
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # IDW merging initialization 
    merger = mergeplg.merge.MergeDifferenceIDW(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        method="additive",
        nnear=nnear,
        range_checks={'diff_check':diff_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"add_p_idw": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenMRG_add_p_idw.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:14<00:00, 50.72it/s]


month: 2015-07


100%|██████████| 744/744 [00:13<00:00, 55.05it/s]


month: 2015-08


100%|██████████| 744/744 [00:13<00:00, 56.43it/s]


In [8]:
# multiplicative IDW
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # IDW merging initialization 
    merger = mergeplg.merge.MergeDifferenceIDW(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        method="multiplicative",
        nnear=nnear,
        range_checks={'ratio_check':ratio_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"mul_p_idw": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenMRG_mul_p_idw.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:12<00:00, 59.50it/s]


month: 2015-07


100%|██████████| 744/744 [00:12<00:00, 58.21it/s]


month: 2015-08


100%|██████████| 744/744 [00:13<00:00, 56.52it/s]


In [9]:
# additive POINT ORDINARY KRIGING
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=False,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="additive",
        range_checks={'diff_check':diff_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"add_p_ok": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenMRG_add_p_ok.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:19<00:00, 37.25it/s] 


month: 2015-07


100%|██████████| 744/744 [00:24<00:00, 30.49it/s] 


month: 2015-08


100%|██████████| 744/744 [00:22<00:00, 33.49it/s]


In [10]:
# multiplicative POINT ORDINARY KRIGING
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=False,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="multiplicative",
        range_checks={'ratio_check':ratio_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"mul_p_ok": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenMRG_mul_p_ok.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:16<00:00, 42.63it/s] 


month: 2015-07


100%|██████████| 744/744 [00:20<00:00, 36.92it/s] 


month: 2015-08


100%|██████████| 744/744 [00:16<00:00, 44.64it/s] 


In [11]:
# additive BLOCK ORDINARY KRIGING
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=True,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="additive",
        range_checks={'diff_check':diff_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"add_b_ok": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenMRG_add_b_ok.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:19<00:00, 36.47it/s]


month: 2015-07


100%|██████████| 744/744 [00:22<00:00, 32.46it/s] 


month: 2015-08


100%|██████████| 744/744 [00:19<00:00, 37.58it/s] 


In [12]:
# multiplicative BLOCK ORDINARY KRIGING
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=True,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="multiplicative",
        range_checks={'ratio_check':ratio_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"mul_b_ok": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenMRG_mul_b_ok.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:16<00:00, 44.11it/s] 


month: 2015-07


100%|██████████| 744/744 [00:19<00:00, 37.77it/s] 


month: 2015-08


100%|██████████| 744/744 [00:16<00:00, 44.86it/s] 


In [13]:
# KED point
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeKrigingExternalDrift(
        ds_rad=ds_rad.rainfall_amount,
        ds_cmls=ds_cmls,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        full_line=False,
        range_checks={'diff_check':diff_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"ked_p": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenMRG_ked_p.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:20<00:00, 35.78it/s]


month: 2015-07


100%|██████████| 744/744 [00:24<00:00, 30.19it/s] 


month: 2015-08


100%|██████████| 744/744 [00:20<00:00, 35.43it/s] 


In [14]:
# KED block
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeKrigingExternalDrift(
        ds_rad=ds_rad.rainfall_amount,
        ds_cmls=ds_cmls,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        full_line=True,
        range_checks={'diff_check':diff_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"ked_b": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenMRG_ked_b.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:19<00:00, 36.71it/s] 


month: 2015-07


100%|██████████| 744/744 [00:24<00:00, 30.51it/s] 


month: 2015-08


100%|██████████| 744/744 [00:19<00:00, 37.33it/s]


# OpenRainER adjustment

In [15]:
# OpenRainER
ds_rad = xr.open_dataset("data/covi_2024_OpenRainER/openrainer_radar.nc")         
ds_cmls = xr.open_dataset("data/processed_cml_OpenRainER.nc").R_acc      
months = ['2022-06', '2022-07', '2022-08']

In [16]:
# methods parameter : default version
nnear = 12 
variogram_parameters = {"sill": 1.0, "range": 30000, "nugget": 0.3}
diff_check_sel = 10
ratio_check_sel = (0.1,15)

In [17]:
# additive IDW
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # IDW merging initialization 
    merger = mergeplg.merge.MergeDifferenceIDW(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        method="additive",
        nnear=nnear,
        range_checks={'diff_check':diff_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"add_p_idw": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenRainER_add_p_idw.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [00:58<00:00, 12.35it/s]


month: 2022-07


100%|██████████| 744/744 [01:02<00:00, 11.92it/s]


month: 2022-08


100%|██████████| 744/744 [01:03<00:00, 11.76it/s]


In [18]:
# multiplicative IDW
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # IDW merging initialization 
    merger = mergeplg.merge.MergeDifferenceIDW(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        method="multiplicative",
        nnear=nnear,
        range_checks={'ratio_check':ratio_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"mul_p_idw": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenRainER_mul_p_idw.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [00:54<00:00, 13.20it/s]


month: 2022-07


100%|██████████| 744/744 [00:54<00:00, 13.59it/s]


month: 2022-08


100%|██████████| 744/744 [01:01<00:00, 12.09it/s]


In [19]:
# additive POINT ORDINARY KRIGING
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=False,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="additive",
        range_checks={'diff_check':diff_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"add_p_ok": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenRainER_add_p_ok.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [08:00<00:00,  1.50it/s]


month: 2022-07


100%|██████████| 744/744 [02:05<00:00,  5.95it/s]


month: 2022-08


100%|██████████| 744/744 [06:14<00:00,  1.99it/s]


In [20]:
# multiplicative POINT ORDINARY KRIGING
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=False,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="multiplicative",
        range_checks={'ratio_check':ratio_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field 
data = xr.Dataset({"mul_p_ok": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenRainER_mul_p_ok.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [04:19<00:00,  2.77it/s]


month: 2022-07


100%|██████████| 744/744 [01:20<00:00,  9.26it/s] 


month: 2022-08


100%|██████████| 744/744 [04:01<00:00,  3.08it/s]


In [21]:
# additive BLOCK ORDINARY KRIGING
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=True,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="additive",
        range_checks={'diff_check':diff_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field 
data = xr.Dataset({"add_b_ok": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenRainER_add_b_ok.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [07:46<00:00,  1.54it/s]


month: 2022-07


100%|██████████| 744/744 [01:58<00:00,  6.27it/s]


month: 2022-08


100%|██████████| 744/744 [05:48<00:00,  2.14it/s]


In [22]:
# multiplicative BLOCK ORDINARY KRIGING
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=True,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="multiplicative",
        range_checks={'ratio_check':ratio_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field  
data = xr.Dataset({"mul_b_ok": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenRainER_mul_b_ok.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [04:07<00:00,  2.90it/s]


month: 2022-07


100%|██████████| 744/744 [01:14<00:00, 10.05it/s] 


month: 2022-08


100%|██████████| 744/744 [03:53<00:00,  3.19it/s]


In [23]:
# KED point
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeKrigingExternalDrift(
        ds_rad=ds_rad.rainfall_amount,
        ds_cmls=ds_cmls,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        full_line=False,
        range_checks={'diff_check':diff_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field 
data = xr.Dataset({"ked_p": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenRainER_ked_p.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [07:56<00:00,  1.51it/s]


month: 2022-07


100%|██████████| 744/744 [02:25<00:00,  5.11it/s]


month: 2022-08


100%|██████████| 744/744 [06:56<00:00,  1.79it/s]


In [24]:
# KED block
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeKrigingExternalDrift(
        ds_rad=ds_rad.rainfall_amount,
        ds_cmls=ds_cmls,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        full_line=True,
        range_checks={'diff_check':diff_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"ked_b": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenRainER_ked_b.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [09:35<00:00,  1.25it/s]


month: 2022-07


100%|██████████| 744/744 [02:05<00:00,  5.94it/s]


month: 2022-08


100%|██████████| 744/744 [06:20<00:00,  1.95it/s]


In [25]:
data = xr.open_dataset('data/adjusted_fields/OpenRainER_ked_b.nc')